In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install nltk


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install re

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: spacy in c:\users\15135\appdata\local\programs\python\python311\lib\site-packages (3.7.2)




[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load data
dfs = pd.read_csv(r'C:\Users\15135\Downloads\instagram.csv')

# Sample about 50% of the data
df = dfs.sample(frac=0.50, random_state=42)

# Load stopwords once, outside the function
english_stopwords = stopwords.words('english')

# Define preprocessing function
def preprocess_text(text, keywords, stopwords):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords]
    if any(keyword in tokens for keyword in keywords):
        return ' '.join(tokens)
    return None

# Define keywords related to updates and fixes
keywords = ['update', 'fix', 'bug', 'issue', 'crash', 'glitch', 'improve', 'patch', 'version']

# Apply the preprocessing function
df['Processed_Review'] = df['review_description'].apply(lambda x: preprocess_text(x, keywords, english_stopwords))
df.dropna(subset=['Processed_Review'], inplace=True)

# Continue with further processing or analysis o

In [7]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')


# Initialize Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Function to get sentiment
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']

# Apply sentiment analysis
df['Sentiment'] = df['review_description'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\15135\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from datetime import datetime

# Load dataset
#df = pd.read_csv(r'C:\Users\15135\Downloads\instagram.csv')

# Sample a smaller subset of the dataset (e.g., 50%)
df_sampled = df.sample(frac=0.5)

# Convert string dates to datetime objects
df_sampled['Review_Date'] = pd.to_datetime(df_sampled['review_date'])

# Convert ratings to a binary classification problem
df_sampled['Sentiment_Class'] = df_sampled['rating'].apply(lambda x: 1 if x > 3 else 0)

# Initialize TF-IDF Vectorizer with reduced features
tfidf = TfidfVectorizer(max_features=500)

# Fit and transform the preprocessed reviews
X_tfidf = tfidf.fit_transform(df_sampled['review_description'].values.astype('U'))

# Define target variable
y = df_sampled['Sentiment_Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize and train the Support Vector Classifier
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

# Predict on the test set and print metrics
y_pred = svc.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
print(f'Precision: {precision_score(y_test, y_pred, average="binary"):.2f}')
print(f'Recall: {recall_score(y_test, y_pred, average="binary"):.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.86
Precision: 0.78
Recall: 0.08
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      2861
           1       0.78      0.08      0.14       492

    accuracy                           0.86      3353
   macro avg       0.82      0.54      0.53      3353
weighted avg       0.85      0.86      0.81      3353



In [9]:
# Define update date and perform analysis
update_date = datetime(2021, 1, 1).date()
df_sampled['Review_Date'] = df_sampled['Review_Date'].dt.date
df_pre_update = df_sampled[df_sampled['Review_Date'] < update_date]
df_post_update = df_sampled[df_sampled['Review_Date'] >= update_date]

# Keywords for Alignment Score
keywords = ['update', 'fix', 'bug', 'improve', 'feature']
keyword_pattern = '|'.join(keywords)

# Calculate Keyword Count using vectorized approach
df_pre_update['Keyword_Count'] = df_pre_update['review_description'].str.count(keyword_pattern)
df_post_update['Keyword_Count'] = df_post_update['review_description'].str.count(keyword_pattern)

# Calculate Alignment Score
alignment_score = df_post_update['Keyword_Count'].mean() - df_pre_update['Keyword_Count'].mean()

# Calculate average sentiment for User Satisfaction
avg_sentiment_pre = df_pre_update['Sentiment_Class'].mean()
avg_sentiment_post = df_post_update['Sentiment_Class'].mean()
satisfaction_change = avg_sentiment_post - avg_sentiment_pre

# Print additional metrics
print(f"Alignment Score: {alignment_score}")
print(f"Average Sentiment Pre-Update: {avg_sentiment_pre}")
print(f"Average Sentiment Post-Update: {avg_sentiment_post}")
print(f"Change in User Satisfaction: {satisfaction_change}")

Alignment Score: -0.014967372699510095
Average Sentiment Pre-Update: 0.17486522911051214
Average Sentiment Post-Update: 0.13585149612116734
Change in User Satisfaction: -0.039013732989344796


C:\Users\15135\AppData\Local\Temp\ipykernel_42868\58423051.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre_update['Keyword_Count'] = df_pre_update['review_description'].str.count(keyword_pattern)
C:\Users\15135\AppData\Local\Temp\ipykernel_42868\58423051.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_post_update['Keyword_Count'] = df_post_update['review_description'].str.count(keyword_pattern)
